In [4]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2LMHeadModel, GPT2Config, AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer_config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model.bin.index.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/special_tokens_map.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/.gitattributes
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.model
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/generation_config.json


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
! pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.7 MB/s eta 0:00:0000:010:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.9 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.6-cp310-cp310-linux_x86_64.whl size=120337372 sha256=2ebbab84027cd3274ab85f6108f5b262ce6b4795e899e7a8bf10b5899df0a6bc
  Stored in directory: /root/.cache/pip/wheels/a8/1c/88/b959d6818b98a46d61ba231683abb7523b89ac1a7ed1e0c206
Successfully built flash-attn


In [7]:
model_path = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model_chat = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [9]:
def generate(prompt):
    messages = [{
        "role":"user",
        "content": prompt
    }]

    model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
    with torch.no_grad():
        generated_ids = model_chat.generate(
            model_inputs,
            max_new_tokens=7500,
            do_sample=True,
            temperature=0.7, 
            top_k=50, 
            top_p=0.95,
            pad_token_id = tokenizer.eos_token_id       
        )

    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    text = decoded[0].split("[/INST]")[1]
    return text

In [18]:
def get_characteristics(product_name, list_of_characteristics):
    prompt = f'Заполни следующие характеристики для товара {product_name}: {list_of_characteristics} на основании наименования этого товара в том случае, если это возможно сделать. Если по наименованию товара невозможно понять значение характеристики, то ставь прочерк вместо значения (-). Выводи в виде "Название характеристики": "Значение характеристики", каждая новая характеристика с новой строки. Каждая характеристики принимает строго одно значение.'
    characteristics = generate(prompt)
    
    return characteristics

In [25]:
product_name = 'Календарь квартальный на 2024 г., 1 блок, 1 гребень, с бегунком, мелованная бумага, BRAUBERG, "Happy Panda", 115312'
list_of_characteristics = 'Тип календаря, Тип скрепления, Внутренний блок, Высота, Год, Дизайн, Индивидуальная упаковка, Календарная сетка, Курсор, Пикколо, Плотность внутреннего блока, Плотность подложки, Подложка, Ширина'

In [26]:
res = get_characteristics(product_name, list_of_characteristics)

In [38]:
res

' Название характеристики: Тип календаря\n"Значение характеристики": Квартальный\n\nНазвание характеристики: Тип скрепления\n"Значение характеристики": Бегунок\n\nНазвание характеристики: Внутренний блок\n"Значение характеристики": 1 блок, 1 гребень\n\nНазвание характеристики: Высота\n"Значение характеристики": -\n\nНазвание характеристики: Год\n"Значение характеристики": 2024 г.\n\nНазвание характеристики: Дизайн\n"Значение характеристики": Happy Panda\n\nНазвание характеристики: Индивидуальная упаковка\n"Значение характеристики": -\n\nНазвание характеристики: Календарная сетка\n"Значение характеристики": -\n\nНазвание характеристики: Курсор\n"Значение характеристики": -\n\nНазвание характеристики: Пикколо\n"Значение характеристики": -\n\nНазвание характеристики: Плотность внутреннего блока\n"Значение характеристики": -\n\nНазвание характеристики: Плотность подложки\n"Значение характеристики": -\n\nНазвание характеристики: Подложка\n"Значение характеристики": -\n\nНазвание характерист